# VENDING MACHINE

## Welcome to my Vending machine!

Please feel free to use it by selecting **Run All** :)

In [ ]:
# Generate data structures and variables

# Declaring global variables
choice = " "
payment = 0
proceed = True
flag = True
sugar = False
mix = False

# Generate data structures
# TODO: MAKE THIS READ FROM FILE INPUT
# TODO: ADD COIN HOLDINGS DICTIONARY
productDict = {"A1": {"Item": "Coffee", "Price": 3.50, "Stock": 5}, "A2": {"Item": "Tea", "Price": 2.00, "Stock": 3}, "B1": {"Item": "Coke", "Price": 4, "Stock": 3}, "B2": {"Item": "Juice", "Price": 4.00, "Stock": 3}}
changeBucket = {2.00: 3, 1.00: 5, 0.50: 4, 0.20: 2, 0.10: 1}

In [ ]:
# Class to handle transactions and record the data of transactions performed

# TODO: Incorporate the tracking of coins inputted and total spent in transactions 
class transaction:
    """ Transaction class for representing and recording transactions performed by the user"""

    global productDict
    global changeBucket

    def __init__(self):
        self.totalPrice = 0
        self.basketDict = {"Coffee": 0, "Tea": 0, "Coke":0, "Juice": 0}
        self.coinEntered = {2.00: 0, 1.00: 0, 0.50: 0, 0.20: 0, 0.10: 0}

    
    def __str__(self):
        return "Basket: {}, Total Price: {}".format(self.basketDict, self.totalPrice)


    def selectItem(self):
        choice = input("Please enter your product code: ")
        return choice

    
    def addItem(self, choice, productDict):
        """ Adds an item to the basket and calculates the new total for that item"""
        prod = productDict[choice]["Item"]
        self.basketDict[prod] = self.basketDict[prod] + 1
        prod = productDict[choice]["Price"]
        self.totalPrice = self.totalPrice + prod


    def getTotalPrice(self):
        """ Returns total price of basket of goods"""
        return self.totalPrice


    def storeTransaction(self):
        pass
        # TODO: Make this write to a new list for data representation
        
    
    def rmvFromInv(self, choice, productDict):
        """ Removes the selected product from the basket of goods and deducts its price from the total"""
        prod = productDict[choice]["Item"]
        self.basketDict[prod] = self.basketDict[prod] - 1
        prod = productDict[choice]["Price"]
        self.totalPrice = self.totalPrice - prod


    def confirmProduct(self, selection):
        """
        Function that asks user to confirm their selection choice and then proceeds.
        """
        print("You have selected: ")
        print(displaySelection(selection))

        print("A    Continue")
        print("B    Cancel Selection")
        confirmation = input("Please select continue if this is correct: ").upper()
        
        if confirmation == "A":
            flag = False

        elif confirmation =="B":
            flag = True
            
        else:
            print("Incorrect Input.")

    
    # Function to handle coin input from user

    def coinInput(self):
        """
        Allow for users to input their coins. Validates that coin is legal tender 
        from list of coins. Shows a total of how much has been paid 
        and how much is still remaining to be paid.
        """

        totPayment = 0
        validCoinInput = [2.00, 1.00, 0.50, 0.20, 0.10]
            
        while totPayment < self.totalPrice:
            payment = float(input("Please deposit your coin: "))

            if payment in validCoinInput: # Checks that legal tender is used
                
                totPayment += payment
                remaining = self.totalPrice - totPayment

                self.coinEntered[payment] = self.coinEntered[payment] + 1

                print("Total Paid: $" + "%.2f" % totPayment)

                if self.totalPrice > totPayment: # Prompt for further payment
                    print("Please insert: $" + "%.2f" % remaining)

                elif self.totalPrice < totPayment: # Calculate using below coinRefund function
                    refund = totPayment - self.totalPrice
                    print("Refund due...")
                    print("Dispensing: $" + "%.2f" % refund)

            else:
                print("Please only enter legal tender.")

        print("Payment processing...")
        print()

    
    def coinRefund(self, totalPrice, totPayment): # TODO: Develop further
        """"""

        pass

    def consume(self):
        """ Remove purchased items from inventory list"""

        for akey in self.basketDict.keys():
            if self.basketDict[akey] > 0:
                for productID, productInfo in productDict.items():
                    for key in productInfo:
                        if productInfo[key] == akey:
                            productInfo["Stock"] = productInfo["Stock"] - 1
      



In [ ]:
# Class to handle data calculations of transaction

class data:
    """ Data class to calculate and present revenue as well as number of particular items sold"""

    revenue = 0
    productsSold = {"Coffee": 0, "Tea": 0, "Coke":0, "Juice": 0}

    def __init__(self): # TODO: Develop further
        pass

    def calculateRevenue(): # TODO: Develop further
        pass
        # TODO: Read from list of data information built in transaction function storeTransaction()

    def numProdSold(): # TODO: Develop further
        pass
        # TODO: Read from list of data information built in transaction function storeTransaction()

    def displayData():
        pass
        # TODO: Display the calculated data from above

In [ ]:
# Function to display nested dictionary of products and keys

def displayProducts():
    """
    Displays Product list and if product is unavailable.
    """
    print("ITEMS AVAILABLE: \n")

    for productID, productInfo in productDict.items(): 
        print("{}    ".format(productID), end="")
        
        for key in productInfo:
            if productInfo[key] == 0: # Message if out of stock
                message = " CURRENTLY NOT AVAILABLE"
            else:
                message = " "
        print(productInfo["Item"] + " $" + str(productInfo["Price"]) + " " + message)



# Display Home page of Vending Machine to allow choice of either user or administrator access

def displayVendingHome():
    """
    Function to display main prompt page.
    """
    print("Please select an option from below.")
    print()
    
    print("A    User Access")
    print("B    Admin Access")
    print("C    Turn Off")


# Displays Product and info

def displaySelection(selection):
    """"
    Displays the selected product and price
    """
    selectedItem = productDict[selection]["Item"]
    selectedItemPrice = productDict[selection]["Price"]

    selectionMsg = "{}    ${}".format(selectedItem, selectedItemPrice)
    return selectionMsg
    
    
# Display option to complete transaction or add more items

def complete():
    """
    Function to ask user if they would like to add any more items or process current basket.
    """

    global flag
    global proceed
    print("Would you like to add more items?")
    displayOption()
    print()
    selection = input("Please enter your choice: ").upper()
    if selection == "A":
        flag = True
        return flag
        
    elif selection == "B":
        proceed = True
        flag = False
        return proceed, flag
        # TODO: if B selected return to main menu
    else:
        print("Incorrect Input")


def displayOption():
    """ Displays YES or NO prior to requesting user input."""

    print("A    Yes")
    print("B    No")


def sugarMixQuery():
    """ 
    Asks for user input as to whether they would like sugar or not.
    Queries if the user would like the sugar automatically mixed or if they wisht to manually mix it.    
    """

    print("Would you like to add sugar?")
    displayOption()
    sugarQuery = input("Please enter your choice: ").upper()

    if sugarQuery == "A":
        sugar = True
        print("Would you like us to mix the sugar for you?")
        displayOption()

        mixQuery = input("Please enter your choice: ").upper()
        if mixQuery == "A":
            mix = True

In [ ]:
# Main Program

while proceed == True:
    displayVendingHome()
    choice = input("Selection: ").upper()
    print()
    if choice == "A":
        print("User Mode Active...")
        print()
        objTransaction = transaction() # New transaction is instanciated when user mode is selected
        print("Good morning and welcome!")
        flag = True # Reset flag to True whenever new transaction initiated

        while flag == True:
            displayProducts()
            selection = objTransaction.selectItem().upper()

            if selection == "A1":
                objTransaction.confirmProduct(selection)
                sugarMixQuery()
                objTransaction.addItem(selection, productDict)
                complete()
                
            elif selection == "A2":
                objTransaction.confirmProduct(selection)
                objTransaction.addItem(selection, productDict)
                complete()

            elif selection == "B1":
                objTransaction.confirmProduct(selection)
                objTransaction.addItem(selection, productDict)
                complete()

            elif selection == "B2":
                objTransaction.confirmProduct(selection)
                objTransaction.addItem(selection, productDict)
                complete()

            else:
                print("Error incorrect input. Please re-enter product code.")

        print(objTransaction)
        objTransaction.coinInput()

        # TODO: Insert refund logic here
        # TODO: Insert print statements around food preperation
        print("Thank you for your purchase!")
        # TODO: Develop function to remove purchased items from inventory
        objTransaction.consume()

    elif choice == "B":
        print("Admin Access Menu:")
        # TODO: ADD SYSTEM STATUS MESSAGE SAYING MACHINE ONLY AVAILABLE FOR ADMINISTRATORS
        # TODO: ADD ADMIN ACCESS MENU AND PROMPTS
        
    elif choice == "C":
        proceed = False

    else:
        print("Incorrect input.")
print("Goodbye...")